# Plotting an SZ signal profile

We plot equation 15 in [Arnaud et al. (2010)](https://ui.adsabs.harvard.edu/abs/2010A%26A...517A..92A/abstract). This paper defines the universal pressure profile (UPP) used in the ACT catalog.

In [ ]:
## Import packages
import numpy as np
from scipy import integrate

import matplotlib.pyplot as plt

## Universal pressure profile

## SZ signal formula

## Plotting the profile